# 環境準備
- 以下の手順で環境を構築する。  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?OSS%E3%81%AELLM
  - WSL2にOllamaをインストールし
  - Ollamaを起動してLlama3を実行させておく。
  - WSL2のNotebookからWebAPI経由でLLMを呼び出す。

# 疎通確認

## HTTPClient
最も単純な、HTTPClient系ライブラリを使用した呼び出し。

### Generate
次単語予測モード

In [ ]:
import requests
url = "http://localhost:11434/api/generate"
payload = {
    "model": "llama3",
    "prompt": "why is the sky blue?"
}

response = requests.post(url, json=payload)
print(response.text) # print(response.json())

### Chat
チャット・モード

In [ ]:
import requests
url = "http://localhost:11434/api/chat"
payload = {
    "model": "llama3",
    "messages": [
        {"role": "user", "content": "why is the sky blue?"}
    ]
}

response = requests.post(url, json=payload)
print(response.text) # print(response.json())

## Ollama
Ollamaライブラリを使用した呼び出し。

### ライブラリのインストール

In [ ]:
!pip install ollama

### ライブラリで呼び出し

#### Generate
次単語予測モード

In [ ]:
import ollama
response = ollama.generate(
    model='llama3',
    prompt='Why is the sky blue?',
)

print(response['response'])

#### Chat
チャット・モード

In [ ]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

# OpenAI
OpenAIライブラリを使用した呼び出し。

### ライブラリのインストール

In [ ]:
!pip install openai

### ライブラリで呼び出し

#### Generate
次単語予測モード

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ASCII code', #ダミーでOK
)

completion = client.completions.create(
    prompt='Why is the sky blue?',
    model='llama3',
)

print(completion.choices[0].text)

#### Chat
チャット・モード

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ASCII code', #ダミーでOK
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': 'Why is the sky blue?',
        }
    ],
    model='llama3',
)

print(chat_completion.choices[0].message.content)

# langchain

In [ ]:
!pip install langchain langchain_core langchain-ollama

# Chatアプリ

## Console & HTTPClient
- 最も単純な、HTTPClient系ライブラリを使用したConsoleChatアプリ
- 以下のコードを修正したもの。  
https://github.com/ollama/ollama/tree/main/examples/python-simplechat

In [ ]:
import json
import requests

model = "llama3"  

def chat(messages):
    r = requests.post(
        "http://localhost:11434/api/chat",
        json={"model": model, "messages": messages, "stream": True},
        # proxies={"http": None, "https": None} #プロキシを無効にするときは必要
    )
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        
        body = json.loads(line)
        
        if "error" in body:
            raise Exception(body["error"])
            
        if body.get("done") is False:
            message = body.get("message", "")
            content = message.get("content", "")
            output += content
            # the response streams one token at a time, print that as we receive it
            print(content, end="", flush=True)

        if body.get("done", False):
            message["content"] = output
            return message

def main():
    messages = []

    while True:
        user_input = input("Enter a prompt: ")
        if not user_input:
            exit()
        print()
        messages.append({"role": "user", "content": user_input})
        message = chat(messages)
        messages.append(message)
        print("\n\n")

if __name__ == "__main__":
    main()

## Streamlit & OpenAI
最も単純な、OpenAIライブラリを使用したConsoleChatアプリ
- Streamlitを使用してWeb化（サーバーサイド実装）
- Stlite（クライアントサイド実装）ではOpenAIが動かないらしい。

### Consoleから以下を実行

In [ ]:
!pip install streamlit

In [ ]:
!streamlit run LLM_Streamlit.py

## ブラウザからアクセスする。  
http://localhost:8501/